# Generador de retroalimentación para su posterior revisión por el docente
Este generador está pensado para pruebas de programación en las que el estudiante sube un solo fichero con su solución. Ejemplos en los que se pide que se implemente una librería, o un parte de un sistema. 

El generador va a iterar sobre todas las soluciones y va a generar un fichero HTML con el mismo nombre que el fichero de la prueba. Este es el primer paso de la coevaluación, en el segundo paso, el docente revisa y modifica la retroalimentación y calificación para poder notificarlo al estudiante.

Se espera que tengas dos carpetas preparadas para ser usadas:

1.- Carpeta **EspacioTrabajo** que contendrá el proyecto completo. En él posteriormente se copiaran la prueba del estudiante una a una y se compilará. Esta carpeta puede ser el esqueleto que se suele dar al estudiante para que trabaje durante el examen.

2.- Carpeta **pruebas** que contendrá todas las pruebas que necesitan retroalimentación. Se asume que es un sólo fichero y que vendrá con el nombre o id de cada estudiante en el nombre. Moodle tiene la opción de descargar todas las pruebas de una tarea.

In [ ]:
# Importamos las liberarías necesarias
import os #Para copiar el código del estudiante al proyecto para compilar
import subprocess #Para compilar el código
from dotenv import load_dotenv #Para cargar las variables de entorno
import anthropic #Para cargar las variables necesarias para la API de Anthropic
from openai import OpenAI   #Para cargar las variables necesarias para la API de OpenAI
load_dotenv()


In [ ]:
# Puedes usar si tienes el LLM de Anthropic
# Añadimos la clave de la API, has debido crearla en OpenIA y almacenarla en el .env del entorno Anaconda.
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

MODEL_CLAUDE="claude-3-5-haiku-20241022"
claude = anthropic.Client(api_key=anthropic_api_key)


In [ ]:
# Para usar el LLM de OpenAI
# Añadimos la clave de la API, has debido crearla en OpenIA y almacenarla en el .env del entorno Anaconda.
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL_GPT = 'o3-mini'
openai = OpenAI()

## Esta parte desglosa el promtp que se va a generar
* **Rúbrica general**: Contendrá las competencias que el docente quiere trabajar durante toda la asignatura (mantenibilidad, eficiencia, funcionalidad, etc).
* **Rúbrica específica**: Desglosará los métodos/funciones, su descripción y sus pesos sobre la calificación de la prueba.
* **Ejemplo de salida**: Tabla HTML que muestra un ejemplo de salida. Es interesante añadir los comentarios de retroalimentación que se esperan, y los que por experiencia, suele ser dónde fallan.

In [ ]:
# Rúbrica general
def rubricaGeneral():
    return """
Eres un evaluador de código de un examen de programación. Tu tarea es analizar la implementación del estudiante y generar una evaluación detallada que incluya una calificación numérica y una retroalimentación formativa para cada método. Para la retroalimentación ten en cuenta:
1. **Mantenimiento:**
 - Evaluar que el código sea fácilmente mantenible y extensible en el futuro.
 - Evitar el uso excesivo de bucles anidados o condiciones complejas.
 - Evitar funciones demasiado largas o complejas.
 - Evitar la duplicación de código.
 - Evitar la creación de variables locales innecesarias.
2. **Claridad:**
 - Asegurar que la estructura del código permita una lectura clara. 
 - Utilizar nombres descriptivos y claros en las variables.
 - Añadir comentarios para documentar partes complejas.
3. **Eficiencia:**
- Evitar llamadas recursivas innecesarias.
- Evitar recorridos innecesarios en estructuras de datos. 
4. **Funcionalidad:**
 - El código debe funcionar en llamadas correctas sin errores.
 - Cuando los parámetros de entrada no estén restringidos, el código de gestionar correctamente la introducción de parámetros incorrectos (por ejemplo valores NULL, número negativos, etc)
**Para la calificación de cada método**. Si la funcionalidad es correcta (punto 4 anterior), tendrá un 100% de la nota del método, si no, tendrá un máximo de un 50%. Puedes restar según los siguientes criterios hasta:
 - Un 10% si el código no es mantenible (punto 1 anterior)
 - Un 10% si el código no es claro (punto 2 anterior)
 - Un 5% si el código no es eficiente (punto 3 anterior)
**Formato de Salida:** La salida de la evaluación debe ser en HTML y contendrá:
 - Una línea con la calificación final sobre 10 y añade <br> al final.
 - Una tabla HTML con tres columnas: "Método", "Puntuación" y "Feedback".
 - Cada fila de la tabla debe corresponder a uno de los ítems evaluados
"""

In [ ]:
def rubricaEspecifica():
    return """Estos son los métodos a evaluar y dar una retroalimentación formativa. En un árbol Splay, siempre que se accede a un nodo (ya sea para borrarlo, modificarlo o leerlo, se debe mover a la raíz):
 - `void insert(K key)` (1 punto): inserta una clave en el árbol. Si la clave ya existe, reemplaza el nodo existente con la nueva clave. Lanza la excepción  **EmptySearchTreeException** si `key` es null.
 - `K search(K key)` (1.5 puntos): busca la clave en el árbol y la devuelve si está presente. Si no se encuentra, devuelve null. Lanza la excepción  **EmptySearchTreeException** si `key` es null.
 - `void delete(K key)` (1 punto): Elimina una clave del árbol si está presente. Lanza la excepción  **EmptySearchTreeException** si `key` es null.
 - `void clear()` (1 punto): Elimina todos los elementos del árbol, dejándolo vacío.
 - `void deleteMinimum()` (0.5 punto): Elimina la clave más pequeña del árbol. Utiliza el método *splay* para mover el mínimo. Lanza la excepción  **EmptySearchTreeException** si el árbol es vacío.
 - `zigzigRight(Node<K> node)` (1 punto): Este método realiza una doble rotación a derecha dado el abuelo (`Node<K> node`). Tras realizar la primera rotación, si el hijo izquierdo es NULL, no realiza la segunda, devolviendo el nodo con una sola rotación aplicada. El nodo de entrada es siempre diferente de NULL.
 - `zigzigLeft(Node<K> node)` (1 punto): Este método realiza una doble rotación a izquierda dado el abuelo (`Node<K> node`). Tras realizar la primera rotación, si el hijo derecho es NULL, no realiza la segunda, devolviendo el nodo con una sola rotación aplicada.El nodo de entrada es siempre diferente de NULL.
 - `zigzagRightLeft(Node<K> node)` (1 punto): Este método realiza una doble rotación, primero a derecha y luego a izquierda dado el abuelo (`Node<K> node`). La primera rotación **no se realiza si no hay subárbol en la rama izquierda del hijo derecho.** El nodo de entrada es siempre diferente de NULL.
 - `zigzagLeftRight(Node<K> node)` (1 punto): Este método realiza una doble rotación, primero a izquierda y luego a derecha dado el abuelo (`Node<K> node`). La primera rotación **no se realiza si no hay subárbol en la rama derecha del hijo izquierdo.** El nodo de entrada es siempre diferente de NULL.
 - `copyOf(SearchTree<K> that)` (0.5 puntos): Realiza una copia de los datos.
 - `copyOf(SplayTree<K> that)` (0.5 puntos): Realiza una copia de los datos manteniendo la misma estructura.

 Los siguientes métodos ya se dan implementados y no se deben evaluar:
 private Node<K> rotateRight(Node<K> node)
 private Node<K> rotateLeft(Node<K> node)
 private Node<K> splay(Node<K> node, K key)
 private Node<K> zigLeft(Node<K> node)
 private Node<K> zigRight(Node<K> node)
 public K minimum()
    """

In [ ]:
def ejemploSalida():
    return """Calificación final: 3/10

<table border="1">
  <tr>
    <th>Método</th>
    <th>Puntuación</th>
    <th>Retroalimentación</th>
  </tr>
  <tr>
    <td>insert(K key)</td>
    <td>0/1</td>
    <td>**Funcionalidad:** El método presenta errores funcionales. Utiliza `search(key) == key`, lo cual compara referencias en lugar de igualdad de valores, esto impide una correcta detección de claves duplicadas. Además, la lógica de inserción dentro del bucle no maneja adecuadamente todos los casos, lo que podría llevar a inserciones incorrectas o a nodos huérfanos. Revisa bien los casos inusuales (un sólo nodo, árbol vacío, etc).
    **Claridad:** Se crean variables innecesarias como `padre` que puede ser eliminadas.
    **Mantenimiento:** Correcto.
    **Eficiencia:** La complejidad del algoritmo de inserción no es óptima (usas varias veces el método *splay*).
    **Sugerencia Formativa:** Revisa bien como funcionan las comparaciones en Java. Siempre prueba casos extremos para probar la robustez del método.
    </td>
  </tr>
  <tr>
    <td>search(K key)</td>
    <td>0/1.5</td>
    <td>**Funcionalidad:** El método no maneja correctamente el caso en que el árbol está vacío, lo que puede causar una excepción `NullPointerException`. Además, utiliza `search(key) == key`, lo cual es incorrecto ya que compara referencias en lugar de igualdad de valores.
    **Claridad:** Las variables tienen nombres descriptivos, lo cual es positivo. Reduce el anidamiento cuando sea posible. Puedes añadir la condición del if negada al bucle y te ahorras el tener un if dentro. Y añade comentarios para facilitar la comprensión del código.
    **Mantenimiento:** El método es corto y hace uso de llamadas a métodos privados, lo cual es positivo. Aunque la complejidad en el bucle con un if anidado dificulta posibles actualizaciones.
    **Eficiencia:** Sin comentarios.
    **Sugerencia Formativa:** Como norma debes revisar que los parámetros en todos los rangos posibles para ver que funciona bien tu código.
    </td>
  </tr>
  <tr>
    <td>delete(K key)</td>
    <td>0/1</td>
    <td>**Funcionalidad:** El método no realiza la eliminación real de la clave en el árbol, deberías poner en nodo a null.
    **Claridad:** La lógica de control es insuficiente para lograr la eliminación, deberías separar la búsqueda (llamar a Splay  y el nodo a eliminar se queda como raiz) de la eliminación.
    **Mantenimiento:** Sin comentarios.
    **Eficiencia:** Sin comentarios.
    **Sugerencia Formativa:** Haz uso del depurador para ver como se comporta tu algoritmo paso a paso y detectar estos errores.
    </td>
  </tr>
  <tr>
    <td>clear()</td>
    <td>1/1</td>
    <td>**Funcionalidad:** Implementación correcta que elimina todos los elementos del árbol.
    **Claridad:** El flujo es claro y lineal, los nombre de las variables son descriptivos.
    **Mantenimiento:** Código fácil de mantener y entender.
    **Eficiencia:** Eficiente en términos de tiempo y espacio.
    **Sugerencia Formativa:** ¡Buena implementación! Clara y sencilla.
      </td>
  </tr>
  <tr>
    <td>deleteMinimum()</td>
    <td>0/0.5</td>
    <td>**Funcionalidad:** El método no maneja correctamente el caso en que el árbol está vacío, lo que debería lanzar una excepción `EmptySearchTreeException`. 
    **Claridad:** Los nombres de las variables deberían ser más descriptivos, y recuerda usar comentarios en las partes más complejas.
    **Mantenimiento:** Rehúsa y no dupliques código. Ya tienes un método Minimum que puedes usar para encontrar el mínimo y otro delete que puedes usar para eliminarlo.
    **Eficiencia:** La implementación es eficiente, aunque la funcionalidad es erronea.
    **Sugerencia Formativa:** Siempre que se pueda, reutiliza código existente para evitar duplicaciones innecesarias.
    </td>
  </tr>
  <tr>
    <td>zigzigRight(Node&lt;K&gt; node)</td>
    <td>1/1</td>
    <td>**Funcionalidad:** La implementación de la doble rotación a la derecha es correcta. 
    **Claridad:** Las variables `nodeToRotate` y `parent` no facilitan un seguimiento claro de las rotaciones. Rehúsa la misma variable para realizar las rotaciones y añade comentarios para explicar el proceso.
    **Mantenimiento:** Correcto, el método es corto y fácil de mantener.
    **Eficiencia:** Eficiente en términos de tiempo O(1).
    **Sugerencia Formativa:** Sigue así, un buen código, eficiente y mantenible.</td>
    </td>
  </tr>
  <tr>
    <td>zigzigLeft(Node&lt;K&gt; node)</td>
    <td>1/1</td>
 <td>**Funcionalidad:** Correcto.
    **Claridad:** Usa nombres de variables más descriptivos y largos, y  añade comentarios para explicar el proceso en las partes más complejas.
    **Mantenimiento:** Correcto.
    **Eficiencia:** Eficiente en términos de tiempo O(1).
    **Sugerencia Formativa:** Una solución clara, eficiente y mantenible.</td>
    </td>
  </tr>
  <tr>
    <td>zigzagRightLeft(Node&lt;K&gt; node)</td>
    <td>0/1</td>
    <td>**Funcionalidad:** La doble rotación en este método no maneja correctamente todos los casos posibles, especialmente cuando no existe un subárbol en la rama izquierda del hijo derecho.
    **Claridad:** Simplifica las condiciones y documenta las partes complejas, facilitando la comprensión del código.
    **Mantenimiento:** La lógica implementada es complicada y propensa a errores, crea métodos privados para apoyarte y facilitar el mantenimiento futuro.
    **Eficiencia:** Sin comentarios.
    **Sugerencia Formativa:** Revisa la lógica de las rotaciones y asegúrate de que se aplican correctamente a los nodos correctos. Puedes usar el depurador para ver como se comporta tu algoritmo paso a paso y detectar estos errores.
    </td>
  </tr>
  <tr>
    <td>zigzagLeftRight(Node&lt;K&gt; node)</td>
    <td>0/1</td>
    <td>**Funcionalidad:** La implementación de la doble rotación izquierda-derecha no gestiona adecuadamente todos los escenarios, especialmente cuando no hay un subárbol en la rama derecha del hijo izquierdo.
    **Claridad:** Simplifica las condiciones y evita el uso de variables innecesarias, complica la legibilidad del código.
    **Mantenimiento:** La lógica implementada es complicada y propensa a errores, crea métodos privados para apoyarte y facilitar el mantenimiento futuro.
    **Eficiencia:** Sin comentarios.
    **Sugerencia Formativa:** Revisa la lógica de las rotaciones y asegúrate de que se aplican correctamente a los nodos correctos. Puedes usar el depurador para ver como se comporta tu algoritmo paso a paso y detectar estos errores.
    </td>
  </tr>
  <tr>
    <td>copyOf(SearchTree&lt;K&gt; that)</td>
    <td>0/0.5</td>
    <td>**Funcionalidad:** El método realiza una copia de los datos pero no mantiene la estructura original del árbol, lo que no cumple con el requisito de mantener la misma estructura.
    **Claridad:** Nombres descriptivos y flujo de ejecución claro.
    **Mantenimiento:** Sin comentarios.
    **Eficiencia:** Sin comentarios.
    **Sugerencia Formativa:** Revisa bien la documentación de la clase y asegúrate de que el método cumple con los requisitos establecidos.
  </td>
  </tr>
  <tr>
    <td>copyOf(SplayTree&lt;K&gt; that)</td>
    <td>0/0.5</td>
    <td>**Funcionalidad:** Similar al método anterior, no preserva la estructura del árbol original, lo que es un incumplimiento de los requisitos.
    **Claridad:** Nombres descriptivos y comentarios acertados.
    **Mantenimiento:** Sin comentarios.
    **Eficiencia:** Sin comentarios.
    **Sugerencia Formativa:** Revisa bien la documentación de la clase y asegúrate de que el método cumple con los requisitos establecidos.
  </td>
  </tr>
</table>
    """       

## El siguiente método se encarga de compilar y devolver el prompt ya generado
* **Rúbrica general**: Contendrá las competencias que el docente quiere trabajar durante toda la asignatura (mantenibilidad, eficiencia, funcionalidad, etc).
* **Rúbrica específica**: Desglosará los métodos/funciones, su descripción y sus pesos sobre la calificación de la prueba.
* **Ejemplo de salida**: Tabla HTML que muestra un ejemplo de salida. Es interesante añadir los comentarios de retroalimentación que se esperan, y los que por experiencia, suele ser dónde fallan.

In [ ]:
def prompt(userCode,compilation_output):
    return f""" {rubricaGeneral()}
                {rubricaEspecifica()}
                {ejemploSalida()}
                Evalua el siguiente código del estudiante.{compilation_output}
                {userSol}"""

In [ ]:
#Compilamos usando mvn
def compilarCodigo(input_file,MODEL):
    dest = "./EspacioTrabajo/src/main/java/org/uma/ed/datastructures/searchtree/SplayTree.java"
    cp_command = f'cp "{input_file}" {dest}'
    subprocess.run(cp_command, shell=True)
    print("Copia realizada")

                                                                       # Command to execute
    command = 'mvn -f ./EspacioTrabajo compile'

    try:
        result = subprocess.run(
            command, shell=True, text=True,capture_output=True
        )
        
        compilation_output = "Compila con ERRORES." if "COMPILATION ERROR" in result.stdout else "Compila correctamente. "
        print(compilation_output)
      
    except Exception as e:
        print("An error occurred while running the compilation command:", str(e))

    userCode = ""
    with open(dest, "r", encoding="utf-8") as infile:
        for line_number, line_content in enumerate(infile, start=1):
            userCode += f"{line_content}"
    return userCode,compilation_output
    

In [ ]:
def readPrueba(input_file,MODEL):
    codigoUsario,salidaCompilacion=compilarCodigo(input_file,MODEL)
    return salidaCompilacion,f"""{rubricaGeneral()}
{rubricaEspecifica()}
{ejemploSalida()}

Evalua el siguiente código del estudiante que {salidaCompilacion}:

{codigoUsario}
    """
    

In [ ]:
def EvaluarPrueba(estudiante,MODEL):
    print("----------------------")
    # Construimos el mensaje para el modelo
    salidaCompilacion, prompt=readPrueba(estudiante, MODEL)
    message = [
        {"role": "user", "content": prompt}
    ]
    # Llamamos al modelo de OpenAI o Anthropic
    result=""
    if (MODEL==MODEL_GPT):
        
        response = openai.chat.completions.create(
            messages=message,
            model=MODEL,
        )
        result= f"{salidaCompilacion}<br>\n" + response.choices[0].message.content
    elif (MODEL==MODEL_CLAUDE):
        response = claude.messages.create(
            model=MODEL_CLAUDE,
            max_tokens=5000,
            temperature=0.1,
            messages=message,
        )
        result = f"{salidaCompilacion}<br>\n" +response.content[0].text 

    return result

## Importante realizar pruebas y ver que la salida del prompt sea correcta
Localiza un examen de un estudiante y haz la prueba a generar el prompt. Luego introduce un error que no permita compilar y vuelve a generar. Así puebas que la generación sea correcta en ambos casos, cuando compila y cuando no.

In [ ]:
def mostrar_con_scroll(texto, alto='300px'):
    html = f"""
    <div style="height: {alto}; overflow: auto; white-space: pre-wrap; font-family: monospace; border: 1px solid #ccc; padding: 10px;">
        {texto}
    </div>
    """
    display(HTML(html))

In [ ]:
from IPython.display import display, HTML
a=readPrueba("./pruebas/prueba1.java",MODEL_GPT)
mostrar_con_scroll(a[1], '500px')

## Otra prueba
Escoge dos o tres estudiantes y observa si consigue calificar bien antes de continuar. Si observas que falta información o que has olvidado algo en alguna rúbrica, ahora es el momento de arreglarlo.


In [ ]:
modelo = MODEL_GPT
#modelo = MODEL_CLAUDE
estudiante = "./pruebas/Senador Rebeca_assignsubmission_file_SplayTree.java"

resultado = EvaluarPrueba(estudiante, modelo)

if (resultado == None):
    print("No se ha podido compilar el código del estudiante. Por favor, revisa el código y vuelve a intentarlo.")
else:
    # Crea el nombre del archivo de salida con extensión .txt
    archivo_resultado = os.path.splitext(estudiante)[0] + "_" + modelo+ "_Resultado.html"
    
    # Guarda el resultado en el archivo .txt
    with open(archivo_resultado, "w") as f:
        f.write(resultado)
        print(f"Resultado guardado en: {archivo_resultado}")

## Llegamos al paso final
Aquí se va iterando sobre todos las pruebas y se solicita el LLM que genere la retroalimentación formativa. Ojo, escoge el modelo que quieres usar. Este se adjunta al nombre por si quieres hacer comparativas.

Observa que se genera un fichero resultados con el prompt de entrada y la salida.

La salida se guarda con extensión HTML y con el mismo nombre que tiene el fichero de pruebas.<>

In [ ]:
import os
import time
# Define la carpeta donde están los archivos .java
carpeta = "./pruebas"
archivo_centralizado = "resultados.txt"
resultados_globales = []
extension = ".java"
# Función para procesar todos los archivos .java
def procesar_pruebas(carpeta, modelo):
    # Recorre todos los archivos en la carpeta
    
    for filename in os.listdir(carpeta):
        print("---------------------------")
        print(filename)
        if filename[0]=="." or not filename.endswith(extension):
            print("Archivo no evaluable ¨" + filename + ", saltamos al siguiente")
            continue
            
        # Solo procesa los archivos con extensión .java
       
        
        estudiante = os.path.join(carpeta + "/", filename )

        # Llama a la función EvaluarExamen y obtén el prompt
        resultado = EvaluarPrueba(estudiante, modelo)

        if (resultado == None):
            print("No se ha podido compilar el código del estudiante. Por favor, revisa el código y vuelve a intentarlo.")
        else:
            resultados_globales.append(f"{estudiante}:{resultado}")
            
            # Crea el nombre del archivo de salida con extensión .txt
            archivo_resultado = os.path.splitext(estudiante)[0] + "_" + modelo+ "_Resultado.html"
            
            # Guarda el resultado en el archivo .txt
            with open(archivo_resultado, "w") as f:
                f.write(resultado)
                print(f"Resultado guardado en: {archivo_resultado}")
            
    with open(archivo_centralizado, "w") as f:
        f.write("\n".join(resultados_globales))
        print(f"Resultados centralizados guardados en: {archivo_centralizado}")
    

# Llamada a la función con el modelo
procesar_pruebas(carpeta, MODEL_GPT)
#procesar_pruebas(carpeta, MODEL_CLAUDE)
